In [ ]:
import numpy as np # <-- Import numpy to use arrays
import pandas as pd # <-- Import pandas to use dataframes
import matplotlib.pyplot as plt # <-- Import matplotlib to plot data

# Exercise 1 - Life cycle assessment of current electricity production

In this exercise, we will calculate the environmental impact of the current electricity production in a fictious country. We will consider the manufacturing and operation of different power plants, as well as the manufacturing of wind turbines and solar panels. We will also consider the treatment of waste from the manufacturing of these products. The environmental impact will be calculated using the ReCiPe method, which includes the following impact categories:

- Climate change
- Human toxicity
- Metals
- Water
- Fine particles emissions

The environmental impact will be calculated per unit of electricity produced by each technology. The results will be presented in stacked bar charts, with each bar representing the contribution of different activities to the total impact in each category.

### List of activities in the system (can also be found in emission_factors.csv)

Make a list called `activities` with the activities found in emission_factors.csv.

## Add activities for manufacturing and operation of power plants

Add activities for the manufacturing and operation of the following power plants:

- Wind turbine
- Solar panel
- Hard coal power plant
- Natural gas power plant
- Hydropower plant

### Add activity for grid electricity

Add an activity for grid electricity to account for the environmental impact of the electricity consumed by the power plants during operation. Call it "grid electricity".

### Create a dictionary to map activities to indices in the matrix

Create a dictionary called `activities` that maps each activity to its index in the matrix. The index should correspond to the position of the activity in the `activities` list.

### For convenience, we create smaller dictionary for each type of activity

Create three dictionaries called `mat`, `build`, and `op` that contain the activities related to materials, manufacturing, and operation, respectively. The keys should be the names of the activities, and the values should be the corresponding indices in the `activities` list. These indices will be used to fill the matrix A with the coefficients of the system.

### Create a dictionary to map direct emissions to indices in the B matrix

Create a dictionary called `direct` that maps each direct emission to its index in the B matrix. The index should correspond to the position of the emission in the list of direct emissions.

### Define the lifetime production of each power plant

Define the lifetime production of each power plant in kWh. The lifetime production is the total amount of electricity produced by the power plant over its lifetime, as described in the powerplants_data.md file. Store the values in a dictionary called `lifetime_production`, where the keys are the names of the power plants, and the values are the lifetime production in kWh.

### Create a matrix A to represent the system, with ones on the diagonal

Create a square matrix A with dimensions equal to the number of activities, using `numpy.identity`. The matrix should have ones on the diagonal and zeros elsewhere. The matrix A will be used to calculate the impact of the system for each activity. Call the matrix `A`.

### Load emission factors and characterization factors

Load the emission factors and characterization factors from the files `emission_factors.csv` and `characterization_factors.csv`, respectively. The emission factors contain the direct emissions to the environment for each activity, while the characterization factors contain the impact of these emissions on the environment. Load the data using `numpy.genfromtxt` and store the results in the variables `ef` and `cf`, respectively.

In [ ]:
ef=np.genfromtxt(
    'emission_factors.csv',
    delimiter=',',
    skip_header=1,
    usecols=range(3, 13)
)

In [ ]:
C=np.genfromtxt(
    'characterization_factors.csv',
    delimiter=',',
    skip_header=1,
    usecols=range(1, 6),
    filling_values=0,
)

### Create a matrix B and include the emission factors

Create a matrix B with dimensions equal to the number of activities and the number of direct emissions. Fill the matrix with the emission factors for each activity. The matrix B will be used to calculate the impact of the system for each direct emission. Call the matrix `B`.

### Fill the matrix A with instructions contained in powerplants_data.md
Input values must be negative, as they represent the consumption of
products from other activities. For example, if the manufacturing of a
wind turbine consumes 30,000 tons of steel, the corresponding entry in
the matrix A should be -30,000. The units of the values in the matrix A
should be consistent with the units of the values in the emission factors
and characterization factors (e.g., for steel, it is kilograms).

### Fill the matrix B with direct emissions to the environment

Fill the matrix B with the direct emissions to the environment for each powerplant. The direct emissions are given in the powerplant_data.md file. The matrix B will be used to calculate the impact of the system for each direct emission.

### Calculate the impact of the system, per technology and store the results in a list

$$
  A \cdot x = f
$$


  - $A$ is the **technology matrix**: a matrix that describes the relationship between processes in the system. Each element $ A_{ij} $ represents how much of product $ i $ is required to produce one unit of product $ j $.
  - $x$ is the **process vector**: the quantity of each process required to fulfill the functional unit.
  - $f$ is the **functional unit vector**, which typically defines the demand for the system output (e.g., how much of the functional unit needs to be produced).

- To solve for $x$, you may use `numpy.linalg.solve`.

- Once $x$ is determined, you can calculate the total environmental flows (inputs/outputs of the entire system).

### Environmental Flow Calculation

- The **inventory vector** $g$ is calculated as:

  $$
  g = B \cdot x
  $$

  - $ B $ is the **flow matrix**: it contains the environmental flows (e.g., energy, materials, emissions) for each process.
  - $ g $ is the vector of cumulative environmental flows (e.g., total emissions) over the entire life cycle.

You will need ensure that the shapes of the matrices allow proper broadcasting and matrix multiplication. You may need to transpose matrices or reshape vectors to ensure the correct shapes for multiplication.


### Life Cycle Impact Assessment (LCIA)
The LCIA phase involves translating the results of the LCI (i.e., the inventory of environmental flows) into potential environmental impacts. This requires the use of characterization factors to translate individual environmental flows into impact categories (e.g., climate change, ozone depletion, human toxicity).


The environmental impact is calculated using characterization factors:
$$
I=C⋅g
$$

- $C$ is the **characterization matrix**: a matrix that contains the characterization factors for each environmental flow and impact category.
- $I$ is the **impact vector**: the impact of the system on each impact category.
- $g$ is the **inventory vector**: the environmental flows (inputs/outputs) of the entire system.


In [ ]:
indicators = [
    "Climate change",
    "Human toxicity",
    "Metals",
    "Water",
    "Fine particle"
]
units = [
    "kg CO2 eq",
    "CTUh",
    "kg antimony eq",
    "m3 water",
    "kg PM 2.5 eq"
]

In [ ]:
activities_colors = {
    "steel": "tab:red",
    "concrete": "tab:red",
    "copper": "tab:red",
    "glass fibre reinforced plastic": "tab:red",
    "neodymium": "tab:red",
    "silicone": "tab:red",
    "glass": "tab:red",
    "aluminium": "tab:red",
    "silver": "tab:red",
    "deionized water": "tab:red",
    "water": "tab:red",
    "hard coal": "tab:orange",
    "natural gas": "tab:orange",
    "concrete waste treatment": "tab:blue",
    "glass waste treatment": "tab:blue",
    'wind turbine (build)': "tab:brown",
    'solar panel (build)': "tab:brown",
    'hard coal power plant (build)': "tab:brown",
    'natural gas power plant (build)': "tab:brown",
    'hydro power plant (build)': "tab:brown",
    "wind turbine (op)": "tab:brown",
    "solar panel (op)": "tab:gray",
    "hard coal power plant (op)": "tab:gray",
    "natural gas power plant (op)": "tab:gray",
    "hydro power plant (op)": "tab:gray",
    "grid electricity": "tab:gray",
}

### Plot the results in stacked bar charts


# Exercise 2 - Apply LCA to scenarios

In this exercise, we will apply the life cycle assessment (LCA) method
 developed above to different scenarios of electricity production.
 
The scenarios are described in scenarios.md, and their corresponding mix is as follows:


In the baseline scenario, the grid slowly transitions toward renewables, but fossil fuels (coal and natural gas) continue to play a major role up to 2050. There is no nuclear energy contribution.


| Year  | Coal (%) | Natural Gas (%) | Hydro (%) | Wind (%) | Solar (%) | Total Renewables (%) |
|-------|----------|-----------------|-----------|----------|-----------|----------------------|
| 2020  | 35       | 30              | 7         | 5        | 4         | 16                   |
| 2025  | 33       | 31              | 7         | 7        | 5         | 19                   |
| 2030  | 30       | 31              | 7         | 10       | 7         | 24                   |
| 2035  | 28       | 30              | 7         | 13       | 9         | 29                   |
| 2040  | 25       | 28              | 7         | 15       | 11        | 33                   |
| 2045  | 20       | 25              | 7         | 18       | 14        | 39                   |
| 2050  | 15       | 22              | 7         | 20       | 16        | 43                   |


In the net-zero scenario, the country rapidly transitions to a fully decarbonized grid, with a strong emphasis on wind, solar, and hydro, and the complete elimination of fossil fuels by 2050.

| Year  | Coal (%) | Natural Gas (%) | Hydro (%) | Wind (%) | Solar (%) | Total Renewables (%) |
|-------|----------|-----------------|-----------|----------|-----------|----------------------|
| 2020  | 35       | 30              | 7         | 5        | 4         | 16                   |
| 2025  | 20       | 25              | 7         | 12       | 9         | 28                   |
| 2030  | 10       | 20              | 8         | 20       | 15        | 43                   |
| 2035  | 5        | 15              | 8         | 25       | 20        | 53                   |
| 2040  | 2        | 10              | 8         | 30       | 25        | 63                   |
| 2045  | 1        | 5               | 8         | 35       | 30        | 73                   |
| 2050  | 0        | 0               | 8         | 40       | 35        | 83                   |


Describe the evolution of the environmental impact of the electricity production system in the two scenarios. Present the results in stacked bar charts, with each bar representing the contribution of different activities to the total impact in each category. Describe the differences between the two scenarios and discuss the implications of the results.


### Modify the matrix A to reflect the new scenarios

Calculate the impact of the system for each scenario for each year and store the results in a list.

### Store the results in a pandas DataFrame and plot the results in stacked bar charts

### Answer the following questions.

1. Does the Net Zero scenario achieve net domestic emissions of zero by 2050? Are there residual emissions in the system? If so, what are the main sources of these emissions?

2. Can you identify trade-offs in the Net Zero scenario?